<a href="https://colab.research.google.com/github/Jgallear/CSSP_brazil_23_24/blob/main/train_on_seq_clusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cartopy

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
from sklearn.inspection import PartialDependenceDisplay
from shapely.geometry import Point
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
#from rasterio import CRS
from shapely.geometry import mapping
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
import functools
import pickle
import cartopy.feature as cf
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
import itertools
from sklearn.preprocessing import MinMaxScaler

In [ ]:
with open('/content/drive/MyDrive/seq_cluster_df_list.pkl', 'rb') as f:
    seq_clusters = pickle.load(f)

In [ ]:
# load df
df = pd.read_csv('/content/drive/MyDrive/VHI_spei_rzsm_dataset.csv')
print(df)

           lon     lat      vhi-1   precip-1      rzsm-1  spei1-1m  spei2-1m  \
0      -53.375 -32.125  70.414172  259.20056   98.021910  2.192168  1.313241   
1      -53.375 -32.125  79.323750  146.58607   98.162980  1.228278  2.453098   
2      -53.375 -32.125  77.111895  184.82845   97.594270  1.472914  1.588110   
3      -53.375 -32.125  64.150561  108.86273   97.605330  1.680747  1.751371   
4      -53.375 -32.125  54.270359  145.90760  100.000000  1.641170  1.900034   
...        ...     ...        ...        ...         ...       ...       ...   
577682 -47.625  -0.875  41.102345  170.38158   66.929490  0.324701  0.572972   
577683 -47.625  -0.875  49.922513  294.35620   80.138200  1.130223  0.881931   
577684 -47.625  -0.875  52.101102  125.48111   88.888840  2.664571  2.250521   
577685 -47.625  -0.875  46.753572   24.13338   77.834076  2.523025  3.002636   
577686 -47.625  -0.875  48.627606   71.58264   69.860950  1.286047  2.275886   

        spei3-1m   spi01-1   spi02-1   

In [ ]:
def groupby_ym(df):
  df_list=[]
  for index,df in df.groupby(['year','month']):
    df_list.append(df)
  return df_list

In [ ]:
def groupby_c(df):
  df_list=[]
  for index,df in df.groupby(['cluster']):
    df_list.append(df)
  return df_list

In [ ]:
print(seq_clusters)
for i in range(len(seq_clusters)):
  seq_clusters[i].drop(columns=['VHI'],inplace=True)

[         lat     lon        VHI  cluster
0    -32.125 -53.375  50.641490        0
1    -31.875 -53.375  51.297176        0
2    -31.875 -53.125  50.021865        0
3    -31.875 -52.875  47.316135        0
4    -31.875 -52.625  47.356915        0
...      ...     ...        ...      ...
2492  -1.125 -48.125  45.059091        0
2493  -1.125 -47.875  43.319721        0
2494  -1.125 -47.625  42.088392        0
2495  -1.125 -47.375  41.638401        0
2496  -0.875 -47.625  43.420835        0

[2497 rows x 4 columns],          lat     lon        VHI  cluster
0    -32.125 -53.375  50.641490        1
1    -31.875 -53.375  51.297176        1
2    -31.875 -53.125  50.021865        1
3    -31.875 -52.875  47.316135        1
4    -31.875 -52.625  47.356915        1
...      ...     ...        ...      ...
2492  -1.125 -48.125  45.059091        0
2493  -1.125 -47.875  43.319721        0
2494  -1.125 -47.625  42.088392        0
2495  -1.125 -47.375  41.638401        0
2496  -0.875 -47.625  43.42083

In [ ]:
print(seq_clusters[0])

         lat     lon  cluster
0    -32.125 -53.375        0
1    -31.875 -53.375        0
2    -31.875 -53.125        0
3    -31.875 -52.875        0
4    -31.875 -52.625        0
...      ...     ...      ...
2492  -1.125 -48.125        0
2493  -1.125 -47.875        0
2494  -1.125 -47.625        0
2495  -1.125 -47.375        0
2496  -0.875 -47.625        0

[2497 rows x 3 columns]


In [ ]:
# need to join clusters df with df, use groupby then loop and merge
#
def join_df_to_clusters(cl_df,df):
  clustered_df_list=[]
  for i in range(len(cl_df)):
    #print(cl_df[i])
    #print(df)
    # need to groupby year and month then merge
    df_list = groupby_ym(df)
    ym_list=[]
    for j in range(len(df_list)):
      df_listj = df_list[j].reset_index()
      # merge with lat and lon
      df_listj2 = df_listj.merge(cl_df[i],on=['lat','lon'])
      ym_list.append(df_listj2)
      # concat
    clustered_i_df = pd.concat(ym_list)
    clustered_df_list.append(clustered_i_df)
  return clustered_df_list

In [ ]:
cl_df_list = join_df_to_clusters(seq_clusters,df)

In [ ]:
print(len(cl_df_list))
print(cl_df_list[2])

10
       index     lon     lat      vhi-1    precip-1     rzsm-1  spei1-1m  \
0       2912 -53.875 -31.375  58.030939  109.296470  24.708150 -1.332931   
1       4161 -52.125 -31.375  44.843541   85.819410  48.470940 -1.207730   
2      11442 -53.375 -30.375  57.982854   60.256737  22.409405 -1.428974   
3      25171 -52.875 -29.125  61.277326   45.327170  21.863266 -1.367052   
4      29540 -52.375 -28.875  54.590231   36.689990  10.798460 -1.424589   
...      ...     ...     ...        ...         ...        ...       ...   
2771  576854 -48.125  -1.125  54.466678  166.388120  81.734795  1.358399   
2772  577062 -47.875  -1.125  52.706162  145.668870  75.352490  1.331064   
2773  577270 -47.625  -1.125  52.049353   96.192610  78.065025  1.270281   
2774  577478 -47.375  -1.125  50.743816   59.039710  68.077020  1.226864   
2775  577686 -47.625  -0.875  48.627606   71.582640  69.860950  1.286047   

      spei2-1m  spei3-1m   spi01-1   spi02-1   spi03-1     pev-1   longrad-1  \
0   

In [ ]:
def split_clusters(df_list):
  clusters_trainx=[]
  clusters_trainy=[]
  clusters_testx=[]
  clusters_testy=[]
  for i in range(len(df_list)):
    dfi = df_list[i]
    train_seti = dfi[dfi['year']<2021]
    test_seti = dfi[dfi['year']>=2021]
    clusters_trainx.append(train_seti.drop(columns=['VHI','index','cluster']))
    clusters_testx.append(test_seti.drop(columns=['VHI','index','cluster']))
    ###
    clusters_trainy.append(train_seti['VHI'])
    clusters_testy.append(test_seti['VHI'])
  return clusters_trainx, clusters_testx, clusters_trainy,clusters_testy

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [ ]:
def GBM_fit_loop(trainxs,testxs,trainys,testys):
  predictions_list=[]
  models=[]
  model_f = GradientBoostingRegressor(n_estimators=200)
  MAE_scores_list=[]
  r2_scores_list=[]
  rmse_scores_list=[]
  for i in range(len(trainxs)):
    print('training...')
    scaler = StandardScaler()
    print('train x')
    print(trainxs[i])
    trainxs_sc = scaler.fit_transform(trainxs[i])
    model_f.fit(trainxs_sc,trainys[i])
    models.append(model_f)
    testxs_sc = scaler.transform(testxs[i])
    predictionsi = model_f.predict(testxs_sc)
    predictions_list.append(predictionsi)
    MAE_scores_list.append(mean_absolute_error(testys[i],predictionsi))
    rmse_scores_list.append(rmse(predictionsi,testys[i]))
    r2_scores_list.append(r2_score(testys[i],predictionsi))
    print('model ', i, ' rmse:',rmse(predictionsi,testys[i]))
    print('model ', i, ' mae:',mean_absolute_error(testys[i],predictionsi))
    print('model ', i, ' r2_score:',r2_score(testys[i],predictionsi))
  return models, predictions_list, testys, np.mean(rmse_scores_list), np.mean(MAE_scores_list), np.mean(r2_scores_list)

In [ ]:
# now need to do split clusters but for each set of clusters
def split_clusters_set(df_list):
  av_rmse_scores_list=[]
  av_mae_scores_list=[]
  av_r2_scores_list=[]
  for i in range(len(df_list)):
    # use split clusters function for each set of clusters
    # need to groupby cluster
    df_cli_list = groupby_c(df_list[i])
    clusters_trainx, clusters_testx, clusters_trainy,clusters_testy = split_clusters(df_cli_list)
    models, predictions_list, testys, av_rmse_score, av_mae_score, av_r2_score = GBM_fit_loop(clusters_trainx, clusters_testx, clusters_trainy,clusters_testy)
    ### list of average error metrics per set of clusters
    av_rmse_scores_list.append(av_rmse_score)
    av_mae_scores_list.append(av_mae_score)
    av_r2_scores_list.append(av_r2_score)
  return av_rmse_scores_list, av_mae_scores_list, av_r2_scores_list

In [ ]:
GBM_model = GradientBoostingRegressor(n_estimators=200)

In [ ]:
av_rmse_scores_list, av_mae_scores_list, av_r2_scores_list = split_clusters_set(cl_df_list)

<ipython-input-33-73cd04db0166>:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for index,df in df.groupby(['cluster']):


training...
train x
         lon     lat      vhi-1    precip-1     rzsm-1  spei1-1m  spei2-1m  \
0    -53.875 -31.375  58.030939  109.296470  24.708150 -1.332931 -1.991701   
1    -52.125 -31.375  44.843541   85.819410  48.470940 -1.207730 -1.632952   
2    -53.375 -30.375  57.982854   60.256737  22.409405 -1.428974 -1.546283   
3    -52.875 -29.125  61.277326   45.327170  21.863266 -1.367052 -1.225164   
4    -52.375 -28.875  54.590231   36.689990  10.798460 -1.424589 -1.263954   
...      ...     ...        ...         ...        ...       ...       ...   
2771 -48.125  -1.125  49.591165  154.066560  47.582700 -0.081858 -0.237530   
2772 -47.875  -1.125  41.426732  152.131840  47.661070 -0.132569 -0.272875   
2773 -47.625  -1.125  37.140970   81.873620  42.565857 -0.104603 -0.232074   
2774 -47.375  -1.125  37.762812   53.660442  43.711160 -0.141673 -0.277371   
2775 -47.625  -0.875  39.895934   68.599335  45.605560 -0.245991 -0.357868   

      spei3-1m   spi01-1   spi02-1   spi03-

<ipython-input-33-73cd04db0166>:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for index,df in df.groupby(['cluster']):


training...
train x
         lon     lat      vhi-1    precip-1     rzsm-1  spei1-1m  spei2-1m  \
246  -50.125 -16.875  31.115492  293.704470  39.038292  1.231933  1.111756   
248  -50.125 -16.625  32.706944  314.170230  45.114210  1.218211  1.079134   
249  -49.875 -16.625  32.331147  324.042100  51.234770  1.189524  1.051921   
256  -51.625 -15.875  31.195499  330.618200  35.718270  1.285612  1.211994   
263  -48.875 -14.375  48.965166  266.993600  57.912650  0.666283  0.533091   
...      ...     ...        ...         ...        ...       ...       ...   
2771 -48.125  -1.125  49.591165  154.066560  47.582700 -0.081858 -0.237530   
2772 -47.875  -1.125  41.426732  152.131840  47.661070 -0.132569 -0.272875   
2773 -47.625  -1.125  37.140970   81.873620  42.565857 -0.104603 -0.232074   
2774 -47.375  -1.125  37.762812   53.660442  43.711160 -0.141673 -0.277371   
2775 -47.625  -0.875  39.895934   68.599335  45.605560 -0.245991 -0.357868   

      spei3-1m   spi01-1   spi02-1   spi03-

<ipython-input-33-73cd04db0166>:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for index,df in df.groupby(['cluster']):


...

training...
train x
         lon     lat      vhi-1    precip-1     rzsm-1  spei1-1m  spei2-1m  \
125  -48.875 -23.875  45.980186   62.280510  17.635406 -0.975709 -0.341206   
126  -48.625 -23.875  49.111731   58.456690  14.895739 -1.011678 -0.341668   
127  -48.125 -23.875  51.588324   48.361774   4.583327 -1.026529 -0.318833   
136  -49.125 -23.625  45.209051   68.515335  14.008612 -1.000380 -0.290483   
137  -48.875 -23.625  47.023794   82.029250  10.710681 -0.955419 -0.290772   
...      ...     ...        ...         ...        ...       ...       ...   
1753 -47.375 -15.875  40.684068  162.145750  32.495216  0.108098 -0.614544   
1754 -47.375 -15.875  40.684068  162.145750  32.495216  0.108098 -0.614544   
1768 -40.375 -15.875  47.990384   82.111450   7.836780  0.322812 -0.740137   
1795 -47.625 -15.625  40.988668  164.450270  31.632618  0.042629 -0.681013   
1806 -42.875 -15.625  36.815390  102.498370  40.398617  0.586426 -0.279799   

      spei3-1m   spi01-1   spi02-1   spi03-

<ipython-input-33-73cd04db0166>:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for index,df in df.groupby(['cluster']):


         lon     lat      vhi-1    precip-1     rzsm-1  spei1-1m  spei2-1m  \
0    -53.875 -31.375  58.030939  109.296470  24.708150 -1.332931 -1.991701   
1    -52.125 -31.375  44.843541   85.819410  48.470940 -1.207730 -1.632952   
2    -53.375 -30.375  57.982854   60.256737  22.409405 -1.428974 -1.546283   
3    -52.875 -29.125  61.277326   45.327170  21.863266 -1.367052 -1.225164   
4    -52.375 -28.875  54.590231   36.689990  10.798460 -1.424589 -1.263954   
...      ...     ...        ...         ...        ...       ...       ...   
693  -50.875 -23.125  40.039010   47.403606   1.718480 -1.832744 -2.735492   
956  -55.375 -21.625  23.280287  205.250020  14.434196 -0.111847 -1.249751   
957  -55.375 -21.625  23.280287  205.250020  14.434196 -0.111847 -1.249751   
1004 -55.125 -21.375  29.866301  201.458710  16.683155 -0.231819 -1.293368   
1005 -55.125 -21.375  29.866301  201.458710  16.683155 -0.231819 -1.293368   

      spei3-1m   spi01-1   spi02-1   spi03-1     pev-1   longra

<ipython-input-33-73cd04db0166>:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for index,df in df.groupby(['cluster']):


training...
train x
         lon     lat      vhi-1    precip-1     rzsm-1  spei1-1m  spei2-1m  \
2411 -44.875 -10.625  53.058542  214.227250   9.088079 -0.876532 -0.325566   
2433 -44.875 -10.375  52.239852  221.876590   7.716172 -0.845185 -0.355260   
2449 -52.375 -10.125  46.767154  427.351560  11.755661 -0.446560 -0.499537   
2484 -50.625  -9.625  53.185865  271.352720   7.570817 -0.957215 -0.381381   
2485 -50.375  -9.625  50.266250  277.154630   6.550026 -0.945489 -0.371560   
...      ...     ...        ...         ...        ...       ...       ...   
2771 -48.125  -1.125  49.591165  154.066560  47.582700 -0.081858 -0.237530   
2772 -47.875  -1.125  41.426732  152.131840  47.661070 -0.132569 -0.272875   
2773 -47.625  -1.125  37.140970   81.873620  42.565857 -0.104603 -0.232074   
2774 -47.375  -1.125  37.762812   53.660442  43.711160 -0.141673 -0.277371   
2775 -47.625  -0.875  39.895934   68.599335  45.605560 -0.245991 -0.357868   

      spei3-1m   spi01-1   spi02-1   spi03-

In [ ]:
no_cl = np.arange(1,10+1)
print(no_cl)

[ 1  2  3  4  5  6  7  8  9 10]


In [ ]:
print(len(no_cl))

10


In [ ]:
print(len(av_rmse_scores_list))
print(av_mae_scores_list)
print(len(av_mae_scores_list))
print(len(av_r2_scores_list))

10
[6.011386334721646, 5.691499881626418, 5.722622407312407, 5.96712755995128, 5.907290562076833, 6.014715633478304, 5.952503993003434, 5.936753834751034, 5.941686371505742, 6.027533633808559]
10
10


In [ ]:
out_df = pd.DataFrame({'mean rmse scores':av_rmse_scores_list,'av_mae_scores_list':av_mae_scores_list,'mean r2 scores':av_r2_scores_list,'number of clusters':no_cl})

In [ ]:
print(out_df)
out_df.to_csv('/content/drive/MyDrive/GBM_model_skill_vs_number_of_clusters.csv',index=False)

   mean rmse scores  av_mae_scores_list  mean r2 scores  number of clusters
0          7.534572            6.011386        0.565574                   1
1          7.027572            5.691500        0.592636                   2
2          7.149719            5.722622        0.543947                   3
3          7.440888            5.967128        0.517238                   4
4          7.370920            5.907291        0.495336                   5
5          7.496290            6.014716        0.447054                   6
6          7.430266            5.952504        0.437836                   7
7          7.492293            5.936754        0.444553                   8
8          7.484598            5.941686        0.411090                   9
9          7.537864            6.027534        0.406769                  10
